# Chapter 4
Christoph Windheuser    
May, 2022   
Python examples of chapter 4 in the book *Learning Spark*


In [1]:
# Import required python spark libraries
import findspark
import pyspark

from pyspark.conf import SparkConf
from pyspark.context import SparkContext

from pyspark.sql.types import *
from pyspark.sql.functions import col, expr, when, concat, lit, avg, desc
from pyspark.sql import SparkSession
from pyspark.sql import Row


In [2]:
# Connect Jupyter Notebook with the Spark application and create Spark Context
findspark.init()
sc = pyspark.SparkContext(appName="chapter_4")


In [3]:
#create a SparkSession

spark = (SparkSession \
         .builder \
         .enableHiveSupport() \
         .config("spark.sql.catalogImplementation","hive") \
         .appName("Chapter_4_Examples") \
         .getOrCreate())


In [4]:
csv_file = "data/departuredelays.csv"

df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))


In [5]:
df.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 5 rows



### Create a View
Create the View from a DataFrame:

In [9]:
df.createOrReplaceTempView("us_delay_flights_view")

Views can be created with SQL from other tables or views.   
In the following example, we create a table with flight delays only from San Francisco (SFO):

In [16]:
spark.sql("""
            CREATE OR REPLACE TEMP VIEW us_delay_flights_SFO_view
            AS SELECT date, delay, distance, origin, destination from us_delay_flights_view
            WHERE origin = 'SFO'
            """)

DataFrame[]

In [17]:
spark.sql("SELECT * FROM us_delay_flights_SFO_view").show(10)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011250|   55|    2247|   SFO|        JFK|
|1012230|    0|    2247|   SFO|        JFK|
|1010705|   -7|    2247|   SFO|        JFK|
|1010620|   -3|    2246|   SFO|        MIA|
|1010915|   -3|     293|   SFO|        LAX|
|1011005|   -8|    1273|   SFO|        DFW|
|1011800|    0|    1604|   SFO|        ORD|
|1011740|   -7|     293|   SFO|        LAX|
|1012015|   -7|     293|   SFO|        LAX|
|1012110|   -1|    2246|   SFO|        MIA|
+-------+-----+--------+------+-----------+
only showing top 10 rows



### Using the view (like a table) in SQL


Show flights with a distance of > 1000 miles and order the results by descendent distance. Show the first 10 results of this list:

In [18]:
spark.sql("""SELECT distance, origin, destination
          FROM us_delay_flights_SFO_view WHERE distance > 1000
          ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
|    2350|   SFO|        BOS|
+--------+------+-----------+
only showing top 10 rows



Instead of spark.sql, the same querry can be executed with the DataFrame API and shows the same result: 

In [19]:
(df.select("distance", "origin", "destination")
   .where("distance > 1000")
   .orderBy("distance", ascending = False).show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



Find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay:

In [20]:
spark.sql("""SELECT date, delay, origin, destination
          FROM us_delay_flights_view
          WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
          ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



Label the flights based on the delays they have experienced. Add a human-readable new column called 'Flight_Delays' containing the labels to the table:

In [21]:
spark.sql("""SELECT delay, origin, destination,
          CASE
              WHEN delay > 360 THEN 'Very Long Delays'
              WHEN delay > 120 AND delay < 360 THEN 'Long Delay'
              WHEN delay > 60 AND delay < 120 THEN 'Short Delay'
              WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delay'
              WHEN delay = 0 THEN 'No Delay'
              ELSE 'Early'
         END AS Flight_Delays
         FROM us_delay_flights_view
         ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|   Long Delay|
|  305|   ABE|        ATL|   Long Delay|
|  275|   ABE|        ATL|   Long Delay|
|  257|   ABE|        ATL|   Long Delay|
|  247|   ABE|        DTW|   Long Delay|
|  247|   ABE|        ATL|   Long Delay|
|  219|   ABE|        ORD|   Long Delay|
|  211|   ABE|        ATL|   Long Delay|
|  197|   ABE|        DTW|   Long Delay|
|  192|   ABE|        ORD|   Long Delay|
+-----+------+-----------+-------------+
only showing top 10 rows



## Creating SQL Tables
(Chapter 4, page 89 ff)

### Create a database called `learn_spark_db`

In [22]:
spark.sql("CREATE DATABASE learn_spark_db")

DataFrame[]

In [23]:
spark.sql("USE learn_spark_db")

DataFrame[]

Spark creates a dictionary `learn_spark_db.db` in the "spark.sql.warehouse.dir" to save tables of the new database.     
The "spark.sql.warehouse.dir" can be get by:

In [24]:
print(spark.conf.get("spark.sql.warehouse.dir"))

file:/home/christoph/Dev/LearningSpark/spark-warehouse


### Create a managed table

A managed table is managed by Spark. It is saved and maintained in a hive repository. When the table is dropped, all content and metadata of the table is deleted.   
To create a managed table, no data source is specified.    
Create a managed table with the SQL API:

In [25]:
spark.sql("""CREATE TABLE managed_us_delay_flights_tbl
             USING CSV
             AS SELECT * FROM us_delay_flights_view""")


DataFrame[]

In [26]:
spark.sql("SELECT * FROM managed_us_delay_flights_tbl").show(10)


+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|3101015|   -5|     352|   DTW|        DCA|
|3100847|   -3|     436|   DTW|        LGA|
|3101200|   -5|     980|   DTW|        FLL|
|3101745|   -3|     832|   DTW|        MCO|
|3101206|   -3|    1452|   DTW|        PHX|
|3101945|   -2|     549|   DTW|        BOS|
|3101950|    5|    1675|   DTW|        SEA|
|3100725|   -3|     995|   DTW|        MIA|
|3100837|   -2|     857|   DTW|        DFW|
|3101950|   -3|    1720|   DTW|        LAX|
+-------+-----+--------+------+-----------+
only showing top 10 rows



Let's drop the table and create it again with the DataFrame API:

In [27]:
spark.sql("DROP TABLE managed_us_delay_flights_tbl")

DataFrame[]

In [28]:
csv_file = "data/departuredelays.csv"
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema = schema)


In [29]:
flights_df.show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|    date| null|    null|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [30]:
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

In [31]:
spark.sql("SELECT * FROM managed_us_delay_flights_tbl").show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01201755|    0|     449|   ORF|        ATL|
|01201610|   52|     449|   ORF|        ATL|
|01201441|    0|     449|   ORF|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



### Viewing the Metadata of databases and tables
With the commands `spark.catalog.` the metadata from Spark cvan be viewed:

In [32]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/home/christoph/Dev/LearningSpark/spark-warehouse'),
 Database(name='learn_spark_db', description='', locationUri='file:/home/christoph/Dev/LearningSpark/spark-warehouse/learn_spark_db.db')]

In [34]:
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_sfo_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='us_delay_flights_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [36]:
spark.catalog.listColumns("managed_us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

Now, let's drop the table again:

In [20]:
spark.sql("DROP TABLE managed_us_delay_flights_tbl")

DataFrame[]

At the end, let's drop the database `learn_spark_db`and all the tables in this database:

In [21]:
spark.sql("DROP DATABASE learn_spark_db CASCADE")

DataFrame[]

### Creating an unmanaged table

Let's first create and use a database:

In [22]:
spark.sql("CREATE DATABASE learn_spark_db")

DataFrame[]

In [23]:
spark.sql("USE learn_spark_db")

DataFrame[]

Now let's create the table `us_delay_flights_tbl` with an SQL command.   
In contrast to create a managed table, here we specify the source of the data for the table.    
Spark will manage the metadata, but not the data of the table.   
If the table is dropped, only the metadata is deleted, but not the data file.

In [24]:
spark.sql("""CREATE TABLE us_delay_flights_tbl
             (date STRING, delay INT, distance INT, origin STRING, destination STRING)
             USING CSV OPTIONS (PATH '/Users/cwi/Dev/LearningSpark/data/departuredelays.csv')""")


DataFrame[]

In [25]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|    date| null|    null|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
+--------+-----+--------+------+-----------+
only showing top 3 rows



Let's drop the table again:

In [26]:
spark.sql("DROP TABLE us_delay_flights_tbl")

DataFrame[]

Now creating the same table with the DataFrame API:   
*(I do not see any differences to the case of creating a* ***managed*** *table with the DataFrame API!)*

In [27]:
csv_file = "data/departuredelays.csv"
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"
flights_df = spark.read.csv(csv_file, schema = schema)


In [28]:
flights_df.show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|    date| null|    null|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [29]:
flights_df.write.saveAsTable("us_delay_flights_tbl")


In [30]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(3)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010630|  -10|     928|   RSW|        EWR|
|01021029|   87|     974|   RSW|        ORD|
|01021346|    0|     928|   RSW|        EWR|
+--------+-----+--------+------+-----------+
only showing top 3 rows



And dropping it again:

In [31]:
spark.sql("DROP TABLE us_delay_flights_tbl")

DataFrame[]

And at the end, let's drop the whole database:

In [32]:
spark.sql("DROP DATABASE learn_spark_db CASCADE")

DataFrame[]

### DataFrameReader
(page 94 ff)

The DataFrameReader is a generic function to read data in different formats into a DataFrame. The following examples are using sample data from the repository https://github.com/databricks/LearningSparkV2. To run these examples, you have to clone the github repo with `git clone https://github.com/databricks/LearningSparkV2.git` 


#### Reading a parquet file

In [40]:
datafile = """../DB_Spark/LearningSparkV2/databricks-datasets/\
learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"""

df = spark.read.format("parquet").load(datafile)

In [41]:
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



#### Reading a CSV file

In [50]:
datafile = """../DB_Spark/LearningSparkV2/databricks-datasets/\
learning-spark-v2/flights/summary-data/csv/*"""

df2 = (spark.read.format("csv")
       .option("inferSchema", "true") \
       .option("header", "true") \
       .option("mode", "PERMISSIVE") \
       .load(datafile))


In [52]:
df2.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



#### Reading a JSON file

In [54]:
datafile = """../DB_Spark/LearningSparkV2/databricks-datasets/\
learning-spark-v2/flights/summary-data/json/*"""

df3 = (spark.read.format("json")
       .load(datafile))

In [55]:
df3.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows

